In [ ]:
from src.trainer import train
import random
import numpy as np
import gymnasium as gym
import config

In [ ]:
agent = train()

Ep 10 reward -5.0 MA(100) 5.80 eps 0.951
Ep 20 reward -9.0 MA(100) 1.15 eps 0.905
Ep 30 reward -10.0 MA(100) -0.03 eps 0.860
Ep 40 reward -11.0 MA(100) -1.68 eps 0.818
Ep 50 reward 1.0 MA(100) -2.54 eps 0.778
Ep 60 reward -6.0 MA(100) -3.62 eps 0.740
Ep 70 reward -10.0 MA(100) -4.54 eps 0.704
Ep 80 reward -8.0 MA(100) -5.10 eps 0.670
Ep 90 reward -10.0 MA(100) -5.59 eps 0.637
Ep 100 reward -10.0 MA(100) -5.85 eps 0.606
q_table stats: min -5.444, mean -0.428, max 1.454
Ep 110 reward -10.0 MA(100) -7.26 eps 0.576
Ep 120 reward -10.0 MA(100) -7.92 eps 0.548
Ep 130 reward -9.0 MA(100) -8.55 eps 0.521
Ep 140 reward -3.0 MA(100) -8.61 eps 0.496
Ep 150 reward -9.0 MA(100) -8.67 eps 0.471
Ep 160 reward -9.0 MA(100) -8.55 eps 0.448
Ep 170 reward -3.0 MA(100) -8.27 eps 0.427
Ep 180 reward -9.0 MA(100) -7.91 eps 0.406
Ep 190 reward -7.0 MA(100) -7.38 eps 0.386
Ep 200 reward -9.0 MA(100) -6.86 eps 0.367
q_table stats: min -12.540, mean -1.367, max 1.635
Ep 210 reward -6.0 MA(100) -6.61 eps 0.349
E

c:\Users\Andrija Lukic\AppData\Local\Programs\Python\Python310\lib\site-packages\pygame\pkgdata.py:27: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Ep 510 reward 102.0 MA(100) 101.67 eps 0.078
Ep 520 reward 106.0 MA(100) 102.15 eps 0.074
Ep 530 reward 112.0 MA(100) 102.82 eps 0.070
Ep 540 reward 100.0 MA(100) 103.19 eps 0.067
Ep 550 reward 101.0 MA(100) 103.55 eps 0.063
Ep 560 reward 113.0 MA(100) 104.25 eps 0.060
Ep 570 reward 106.0 MA(100) 104.63 eps 0.057
Ep 580 reward 105.0 MA(100) 105.40 eps 0.055
Ep 590 reward 108.0 MA(100) 106.16 eps 0.052
Ep 600 reward 104.0 MA(100) 106.92 eps 0.049
q_table stats: min -13.698, mean 14.215, max 77.770
Ep 610 reward 106.0 MA(100) 107.31 eps 0.047
Ep 620 reward 104.0 MA(100) 108.05 eps 0.045
Ep 630 reward 108.0 MA(100) 108.46 eps 0.043
Ep 640 reward 106.0 MA(100) 108.86 eps 0.040
Ep 650 reward 102.0 MA(100) 109.02 eps 0.038
Ep 660 reward 110.0 MA(100) 109.19 eps 0.037
Ep 670 reward 106.0 MA(100) 109.86 eps 0.035
Ep 680 reward 113.0 MA(100) 110.03 eps 0.033
Ep 690 reward 112.0 MA(100) 110.42 eps 0.031
Ep 700 reward 114.0 MA(100) 110.68 eps 0.030
q_table stats: min -13.506, mean 17.689, max 93.

In [ ]:
def test(fql_agent, num_episodes=100, render=False, seed=None, max_steps=None, verbose=True):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    env = gym.make(config.ENV_NAME)
    max_steps = max_steps or getattr(config, "MAX_STEPS_PER_EPISODE", 1000)

    prev_eps = getattr(fql_agent, "epsilon", None)
    if prev_eps is not None:
        fql_agent.epsilon = 0.0

    episode_rewards = []
    try:
        for ep in range(1, num_episodes + 1):
            if seed is not None:
                obs, _ = env.reset(seed=seed + ep)
            else:
                obs, _ = env.reset()

            total_reward = 0.0
            terminated = truncated = False
            steps = 0
            cart_pos, cart_vel, pole_angle, pole_ang_vel = obs
            action = fql_agent.get_initial_action([float(pole_angle), float(pole_ang_vel)])

            while not (terminated or truncated) and steps < max_steps:
                obs, reward, terminated, truncated, _ = env.step(int(action))
                total_reward += reward
                steps += 1

                if terminated or truncated:
                    break

                _, _, pole_angle, pole_ang_vel = obs
                action = fql_agent.get_action([float(pole_angle), float(pole_ang_vel)])

            episode_rewards.append(total_reward)

            if verbose and (ep == 1 or ep % 10 == 0 or ep == num_episodes):
                recent_ma = float(np.mean(episode_rewards[-100:]))
                print(f"[Test] Ep {ep}/{num_episodes} reward {total_reward:.1f} recent_MA({min(100,len(episode_rewards))}) {recent_ma:.2f}")

    finally:
        if prev_eps is not None:
            fql_agent.epsilon = prev_eps
        env.close()

    stats = {
        "mean": float(np.mean(episode_rewards)),
        "std": float(np.std(episode_rewards)),
        "min": float(np.min(episode_rewards)),
        "max": float(np.max(episode_rewards)),
        "median": float(np.median(episode_rewards)),
    }

    if verbose:
        print("=== Evaluation summary ===")
        print(f"episodes: {len(episode_rewards)}  mean: {stats['mean']:.2f}  std: {stats['std']:.2f}  min: {stats['min']:.1f}  max: {stats['max']:.1f}")

    return episode_rewards, stats

In [7]:
test_rewards, test_stats = test(agent[0], num_episodes=50, render=True, seed=42)

[Test] Ep 1/50 reward 500.0 recent_MA(1) 500.00
[Test] Ep 10/50 reward 500.0 recent_MA(10) 500.00
[Test] Ep 20/50 reward 500.0 recent_MA(20) 500.00
[Test] Ep 30/50 reward 500.0 recent_MA(30) 500.00
[Test] Ep 40/50 reward 500.0 recent_MA(40) 499.43
[Test] Ep 50/50 reward 500.0 recent_MA(50) 499.54
=== Evaluation summary ===
episodes: 50  mean: 499.54  std: 3.22  min: 477.0  max: 500.0


In [5]:
np.save(f"q_table_best_ep{2700}.npy", agent[0].q_table)